In [3]:
import openai
import time
# openai.api_key = 's__k-73ZF1jNfuykC7q3L1wL4T3BlbkFJRRPFp17Qlg9Y9eaFJ32___r'


In [4]:
topic    = "acquisition of company named XYZ"

In [10]:
def parse_text(text):
    sub_topics = text.strip().split("\n")
    sub_topics_list = [sub_topic.strip().split(".", 1)[1].strip() for sub_topic in sub_topics if sub_topic]
    return sub_topics_list

def get_subtopics(topic):
    additional_infos_prompt = f"give me 5 sub-topics related to {topic} without explanations and be generic. do not mention company names."
    print(additional_infos_prompt)
    res_additional_prompt = openai.Completion.create(
      engine="text-davinci-003",
      prompt=additional_infos_prompt,
      max_tokens=2000,
      temperature=0.7,
      stop=None,
    )
    additions = parse_text(res_additional_prompt['choices'][0]['text'])
#     print(additions)
    return additions

In [11]:
additions = [ x for x in get_subtopics(topic) if x != '']

give me 5 sub-topics related to acquisition of company named XYZ without explanations and be generic. do not mention company names.
['Mergers & Acquisitions', 'Due Diligence', 'Integration Planning', 'Financing Structures', 'Regulatory Requirements']


In [12]:
additions

['Mergers & Acquisitions',
 'Due Diligence',
 'Integration Planning',
 'Financing Structures',
 'Regulatory Requirements']

In [15]:
additions.__len__()

5

In [ ]:
all_res = []
add_ind = 0
tries = 0
while (add_ind < additions.__len__()) and tries<10:
    tries += 1
    addition = additions[add_ind]
    source   = "an internal corporate email"
    N_per_request = 10
    prompt =  f"""write a sentence or a short paragraph about the {addition} with regard to {topic}. 
              assume the content is taken from {source}"""
    print(prompt)
    try:
        res = openai.Completion.create(
          engine="text-davinci-003",
          prompt=prompt,
          max_tokens=2000,
          temperature=0.7,
          n=N_per_request,
          presence_penalty=1.0,
          stop=None,
        )
    except Exception as e:
        print( "=======failed========")
        print(e)
        time.sleep(60)
        continue
        
    add_ind += 1
    print([x['text'] for x in res['choices']])
    all_res.append([x['text'] for x in res['choices']])
    time.sleep(1)
    

write a sentence or a short paragraph about the Mergers & Acquisitions with regard to acquisition of company named XYZ. 
              assume the content is taken from an internal corporate email
["\n\nXYZ has been a major player in the market and their acquisition would be an important step in our growth strategy. We are confident that this merger will bring us significant competitive advantages, both in terms of cost savings and operational efficiencies. This acquisition will also give us access to XYZ's customer base and technology, which will enable us to increase our market share and presence. We look forward to revealing more details about this exciting partnership in the coming weeks.", '\n\nWe are pleased to announce that our company has successfully completed the acquisition of XYZ. This merger and acquisition will provide us with new opportunities for growth, as well as a stronger foothold in the industry. We look forward to leveraging the resources of both companies to furth

In [17]:
from openpyxl import Workbook

import openpyxl
book = openpyxl.load_workbook('data.xlsx')
worksheet = book.active
for addition_ind, addition_res in enumerate(all_res):
    for text in addition_res:
        row = [topic, additions[addition_ind], text]
        worksheet.append(row)

book.save("data.xlsx")
